In [2]:
import cv2
import matplotlib.pyplot as plt
import skimage.io as imageio
import numpy as np
import pandas as pd
from tensorflow import keras


In [3]:
# load model 
model= keras.models.load_model("drowsiness_model-94")


In [28]:
# load haarcascade_eye
eyedetect=cv2.CascadeClassifier('haarcascade_eye.xml')

In [29]:
# fun to plt the two eye in each frame
def show_eyes(e):
    plt_num = len(e)
    fig, ax = plt.subplots(nrows = 1, ncols = plt_num, figsize = (20,20))
    for i in range(0, plt_num):
        ax[i].imshow(e[i])
        ax[i].axis('off')

# fun to predict two eyes in each frame and return 0 if two eye is close         
def  predict_model(e):

    #resize first eye to fit to model inputs
    eye1=np.array(cv2.resize(e[0],(128,128)))
    eye1=np.expand_dims(eye1,axis=0)
    #resize second eye to fit to model inputs
    eye2=np.array(cv2.resize(e[1],(128,128)))
    eye2=np.expand_dims(eye2,axis=0)
  
    
    # call predict model and return 1 if open , 0 if close
    left_eye_predict = model.predict(eye1)
    right_eye_predict = model.predict(eye2)
    
    # to around and convert value of predeict to integr,, if value >=0.5 convert it 1 else 0 
    left_eye_predict=int(np.around(left_eye_predict))
    right_eye_predict= int(np.around(right_eye_predict))
    
    # (1,1) (1,0) (0,1) (0,0)
    print(left_eye_predict, right_eye_predict)
    
    # 2 eye open
    if left_eye_predict and right_eye_predict:
        return 1
    # one open and one close
    elif (left_eye_predict and not right_eye_predict) or (not left_eye_predict and right_eye_predict):
        return 2
    # 2 eye close
    else:
        return 0
    

In [30]:
# open live video
vs = cv2.VideoCapture(0)
eyes=[]
while vs.isOpened():
    #read video
    r, frame = vs.read()
    # convert frame to rgb
    rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    # detect two eyes in each frame
    geteye=eyedetect.detectMultiScale(rgb)
    # list to add two eyes in each frame
    gozler=[]
    if (len(geteye)==2): 
        for (x,y,w,h) in geteye:
            gozler.append(rgb[y:y+h, x:x+w])
            eyes.append(rgb[y:y+h, x:x+w])
        result=predict_model(gozler)    
        if result ==1:
            print('two eyes open')
        elif result==2 :
            print('one open and one close')
        elif result==0:
            print('two eyes close')
        
    
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
        
        
vs.release()
cv2.destroyAllWindows()



1 1
two eyes open
1 1
two eyes open
1 1
two eyes open
1 1
two eyes open
1 1
two eyes open
1 1
two eyes open
1 1
two eyes open
1 1
two eyes open
1 1
two eyes open
1 1
two eyes open
0 0
two eyes close
0 0
two eyes close
0 0
two eyes close
0 0
two eyes close
0 0
two eyes close
0 0
two eyes close
0 0
two eyes close
1 0
one open and one close
1 0
one open and one close
0 0
two eyes close
1 1
two eyes open
1 1
two eyes open
1 1
two eyes open
0 0
two eyes close
0 0
two eyes close
0 0
two eyes close
1 1
two eyes open
1 1
two eyes open
1 1
two eyes open
0 1
one open and one close
0 0
two eyes close
0 0
two eyes close
0 1
one open and one close
1 0
one open and one close


In [31]:
show_eyes(eyes)